In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv("dataset.csv", delimiter=';')
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
X = df.drop(columns=['date', 'product', 'malfunction'])
y = df['malfunction']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Imbalance ratio for XGBoost
scale_ratio = y_train.value_counts()[0] / y_train.value_counts()[1]

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(
        use_label_encoder=False,
        eval_metric='aucpr',
        scale_pos_weight=scale_ratio,
        max_depth=4,
        learning_rate=0.02,
        n_estimators=200,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
}

# Run and collect results
results = {}

for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)
    probs = model.predict_proba(X_test_scaled)[:, 1]

    # Dynamically find threshold with best F1-score
    precisions, recalls, thresholds = precision_recall_curve(y_test, probs)
    f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 0.000001)
    best_idx = f1_scores.argmax()
    best_threshold = thresholds[best_idx]
    y_pred = (probs >= best_threshold).astype(int)

    report = classification_report(y_test, y_pred, digits=4)
    matrix = confusion_matrix(y_test, y_pred)

    results[name] = {
        "threshold": best_threshold,
        "f1_score": f1_scores[best_idx],
        "recall": recalls[best_idx],
        "precision": precisions[best_idx],
        "report": report,
        "conf_matrix": matrix
    }

    print(f"\n🔍 {name}")
    print(f"Best Threshold: {best_threshold:.4f}")
    print(f"Precision: {precisions[best_idx]:.4f}")
    print(f"Recall:    {recalls[best_idx]:.4f}")
    print(f"F1 Score:  {f1_scores[best_idx]:.4f}")
    print("Classification Report:")
    print(report)
    print("Confusion Matrix:")
    print(matrix)

# Summary for XGBoost
print("\n✅ XGBoost Evaluation (Optimized for Balanced F1 and Recall)")
print(f"Best Threshold: {results['XGBoost']['threshold']:.4f}")
print(f"Precision: {results['XGBoost']['precision']:.4f}")
print(f"Recall:    {results['XGBoost']['recall']:.4f}")
print(f"F1 Score:  {results['XGBoost']['f1_score']:.4f}")
print("Classification Report:")
print(results["XGBoost"]["report"])
print("Confusion Matrix:")
print(results["XGBoost"]["conf_matrix"])




🔍 Logistic Regression
Best Threshold: 1.0000
Precision: 0.1875
Recall:    0.1429
F1 Score:  0.1622
Classification Report:
              precision    recall  f1-score   support

           0     0.9993    0.9995    0.9994     24878
           1     0.1875    0.1429    0.1622        21

    accuracy                         0.9988     24899
   macro avg     0.5934    0.5712    0.5808     24899
weighted avg     0.9986    0.9988    0.9987     24899

Confusion Matrix:
[[24865    13]
 [   18     3]]

🔍 Decision Tree
Best Threshold: 1.0000
Precision: 0.0882
Recall:    0.1429
F1 Score:  0.1091
Classification Report:
              precision    recall  f1-score   support

           0     0.9993    0.9988    0.9990     24878
           1     0.0882    0.1429    0.1091        21

    accuracy                         0.9980     24899
   macro avg     0.5438    0.5708    0.5541     24899
weighted avg     0.9985    0.9980    0.9983     24899

Confusion Matrix:
[[24847    31]
 [   18     3]]

🔍 Rando

C:\Users\ASUS\anaconda3\lib\site-packages\xgboost\core.py:158: UserWarning: [20:42:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



🔍 XGBoost
Best Threshold: 0.9977
Precision: 0.0909
Recall:    0.0952
F1 Score:  0.0930
Classification Report:
              precision    recall  f1-score   support

           0     0.9992    0.9992    0.9992     24878
           1     0.0909    0.0952    0.0930        21

    accuracy                         0.9984     24899
   macro avg     0.5451    0.5472    0.5461     24899
weighted avg     0.9985    0.9984    0.9985     24899

Confusion Matrix:
[[24858    20]
 [   19     2]]

✅ XGBoost Evaluation (Optimized for Balanced F1 and Recall)
Best Threshold: 0.9977
Precision: 0.0909
Recall:    0.0952
F1 Score:  0.0930
Classification Report:
              precision    recall  f1-score   support

           0     0.9992    0.9992    0.9992     24878
           1     0.0909    0.0952    0.0930        21

    accuracy                         0.9984     24899
   macro avg     0.5451    0.5472    0.5461     24899
weighted avg     0.9985    0.9984    0.9985     24899

Confusion Matrix:
[[24858